In [1]:
termsToSearch = ['rebate','calculation','home']
searchPhrases = ['home sale referral rebate','service fee','calculation of rebate','rebate']

In [2]:
def contractWordSearch(searchTerm):

    import os
    import pandas as pd
    #import re
    from textblob import TextBlob

    path = 'C:/Users/jrstrong/Desktop/GSM/SOO/BI Projects/ContractMining/Contracts/'
    #path = '//orsraptorweb01.amr.corp.intel.com/ContractAnalytics/Converted_Documents/ConvertedTXT/'

    folder = os.fsencode(path)

    filenames = []

    for file in os.listdir(folder):
        filename = os.fsdecode(file)
        if filename.endswith( ('.txt') ): # whatever file types you're using...
            filenames.append(filename)

    filenames.sort() # now you have the filenames and can do something with them
    
    dfSearchTerms = pd.DataFrame(columns=['Doc','SearchTermCnt'])
    
    for n in filenames:
        termCnt = {}
        termCnt.clear()
        file = str(n)
        file = file.replace("['","")
        file = file.replace("']","")
        filepath = path + file
        file_object = open(filepath, errors="ignore")
        contract = file_object.read()
        blob = TextBlob(contract)
        for term in searchTerm:
            termCnt[term] = blob.word_counts[term]
        
        
        dfSearchTerms = dfSearchTerms.append(pd.DataFrame(data = {'Doc':file.replace(".txt",""), 'SearchTermCnt':[termCnt]}),ignore_index=True)
        
    return dfSearchTerms

In [27]:
def contractPhraseSearch (searchPhrase):
    import os
    import pandas as pd
    import re
    
    #path = 'C:/Users/jrstrong/Desktop/GSM/SOO/BI Projects/ContractMining/Contracts/'
    path = '//orsraptorweb01.amr.corp.intel.com/ContractAnalytics/Converted_Documents/ConvertedTXT/'

    folder = os.fsencode(path)

    filenames = []

    for file in os.listdir(folder):
        filename = os.fsdecode(file)
        if filename.endswith( ('.txt') ): # whatever file types you're using...
            filenames.append(filename)

    filenames.sort() # now you have the filenames and can do something with them
    
    dfSearchTerms = pd.DataFrame(columns=['Doc','SearchTermCnt'])
    
    printCounter = 0
    printCounterTotal = 0
    
    for n in filenames:
        termCnt = {}
        termCnt.clear()
        file = str(n)
        file = file.replace("['","")
        file = file.replace("']","")
        filepath = path + file
        file_object = open(filepath, errors="ignore")
        contract = file_object.read()
        contract = contract.lower()
        for term in searchPhrase:
            termCnt[term] = len([m.start() for m in re.finditer(r'\b' + term + r'\b', contract)])
            
        printCounter += 1
        if (printCounter == 1000):
            print('Completed ' + str(printCounterTotal) + ' documents. (' + str("{:.2%}".format(printCounterTotal/len(filenames))) +')')
            printCounterTotal += printCounter
            printCounter = 0
        
        
        dfSearchTerms = dfSearchTerms.append(pd.DataFrame(data = {'Doc':file.replace(".txt",""), 'SearchTermCnt':[termCnt]}),ignore_index=True)
        
    return dfSearchTerms

In [4]:
def unpack(df, column, fillna=None):
    import pandas as pd
    ret = None
    if fillna is None:
        ret = pd.concat([df, pd.DataFrame((d for idx, d in df[column].iteritems()))], axis=1)
        del ret[column]
    else:
        ret = pd.concat([df, pd.DataFrame((d for idx, d in df[column].iteritems())).fillna(fillna)], axis=1)
        del ret[column]
    return ret

In [27]:
dfSearchTerms = unpack(contractWordSearch(termsToSearch), 'SearchTermCnt', 0)
dfSearchTerms['TotalCnt'] = 0
dfSearchTerms['TotalCnt'] = dfSearchTerms.drop('Doc', 1).sum(axis=1).rename('TotalCnt')
dfSearchTerms = dfSearchTerms.loc[dfSearchTerms['TotalCnt'] > 0]

In [28]:
dfSearchTerms

,Doc,calculation,home,rebate,TotalCnt
7,Doc7216830,0,0,2,2
18,Doc7314073,0,0,2,2
19,Doc7314091,0,0,2,2
20,Doc7314093,0,2,0,2
23,Doc7318589,0,2,0,2
24,Doc7318594,0,0,2,2
25,Doc7318668,0,0,2,2
27,Doc7386760,3,0,0,3
28,Doc7425054,0,0,2,2
32,Doc7451699,0,0,2,2


In [29]:
dfPhraseSearch = unpack(contractPhraseSearch (searchPhrases), 'SearchTermCnt', 0)
dfPhraseSearch['TotalCnt'] = 0
dfPhraseSearch['TotalCnt'] = dfSearchTerms.drop('Doc', 1).sum(axis=1).rename('TotalCnt')
dfPhraseSearch = dfSearchTerms.loc[dfSearchTerms['TotalCnt'] > 0]

Completed 0 documents. (0.00%)
Completed 1000 documents. (1.34%)
Completed 2000 documents. (2.69%)
Completed 3000 documents. (4.03%)
Completed 4000 documents. (5.37%)
Completed 5000 documents. (6.71%)
Completed 6000 documents. (8.06%)
Completed 7000 documents. (9.40%)
Completed 8000 documents. (10.74%)
Completed 9000 documents. (12.08%)
Completed 10000 documents. (13.43%)
Completed 11000 documents. (14.77%)
Completed 12000 documents. (16.11%)
Completed 13000 documents. (17.45%)
Completed 14000 documents. (18.80%)
Completed 15000 documents. (20.14%)
Completed 16000 documents. (21.48%)
Completed 17000 documents. (22.82%)
Completed 18000 documents. (24.17%)
Completed 19000 documents. (25.51%)
Completed 20000 documents. (26.85%)
Completed 21000 documents. (28.19%)
Completed 22000 documents. (29.54%)
Completed 23000 documents. (30.88%)
Completed 24000 documents. (32.22%)
Completed 25000 documents. (33.56%)
Completed 26000 documents. (34.91%)
Completed 27000 documents. (36.25%)
Completed 280

NameError: name 'dfSearchTerms' is not defined

In [30]:
dfPhraseSearch

,Doc,calculation of rebate,home sale referral rebate,rebate,service fee,TotalCnt
0,Doc1000015244,0,0,0,0,0
1,Doc1000015251,0,0,0,0,0
2,Doc1000015254,0,0,0,0,0
3,Doc1000029061,0,0,0,0,0
4,Doc1000037081,0,0,0,0,0
5,Doc1000037087,0,0,0,0,0
6,Doc1000037089,0,0,0,0,0
7,Doc1000037092,0,0,0,0,0
8,Doc10000426,0,0,1,0,0
9,Doc10000432,0,0,2,3,0


In [31]:
dfPhraseSearch.to_csv('C:/Users/jrstrong/Desktop/GSM/SOO/BI Projects/ContractMining/Contracts/InboundPayment.csv')